In [1]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
import h5py
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Documents/Science/MirRevAdaptEEG'
participants = list(range(0,32))
#specify which erp we are analyzing
erps = 'frn'
#specify electrodes to use for plotting (all, frontal, central, parietal, visual)
roi = 'latcen'

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [2]:
#setting up path/ directory
#access trial type epoched data for each participant
def load_tfr_epochs(pp_num, root_dir, erp_path, task):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/')
    id_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    pp_directory = os.path.join(id_directory, erp_path)
    filename = os.path.join(pp_directory, 'p%03d_%s-epo.fif' % (pp_num, task))

    epochs = mne.read_epochs(filename)

    return epochs, pp_directory

In [3]:
#set up parameters for time-frequency morlet convolution
def set_tfr_params(l_freq = 6, h_freq = 35, num_freq = 50, cycle = 6, samp_rate = 200):
    freqs = np.logspace(*np.log10([l_freq, h_freq]), num = num_freq)
    n_cycles = cycle
    sfreq = samp_rate
    
    return freqs, n_cycles, sfreq

In [4]:
#save tfr'd data
def output_tfr_data(root_dir, data, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/tfr_ana/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s-tfr.h5' % (task, output))
    mne.time_frequency.write_tfrs(out_fname, tfr=data, overwrite = True)

In [5]:
#load tfr'd data
def load_tfr_data(root_dir, task, output):
    # Save the tfr'd data
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/eeg/tfr_ana/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s-tfr.h5' % (task, output))
    dat = mne.time_frequency.read_tfrs(out_fname)
    
    return dat

In [6]:
# save tfr plot
def save_tfr_plot(root_dir, fig, task, output, roi):
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'doc/fig/tfr_mne/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s_%s-tfr.svg' % (roi, task, output))
    fig.set_size_inches(8,6)
    fig.savefig(out_fname)
    matplotlib.pyplot.close(fig)

In [ ]:
# save tfr channel plot
def save_tfr_channel_plot(root_dir, fig, task, output, channel):
    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'doc/fig/tfr_mne/%s' % (erps))
    out_fname = os.path.join(data_directory, '%s_%s_%s-tfr.svg' % (channel, task, output))
    fig.set_size_inches(8,6)
    fig.savefig(out_fname)
    matplotlib.pyplot.close(fig)

In [ ]:
# save average tfr across participants for each condition
# #morlet convolution for each participant, according to trial types - POWER

# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# for pcond in range(0, len(perturb_conds)):
#     pp_power = []
#     for pp in participants:
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = False
#         )
#         #average epochs within participant first, then across participants
#         win_power = power.average()
#         pp_power.append(win_power)
#     #grand average of tfr across participants    
#     alldat = mne.grand_average(pp_power, interpolate_bads=True, drop_bads=True)
#     #save tfr object
#     output_tfr_data(root_dir = root, data = alldat, task = perturb_conds[pcond], output = "power")

In [ ]:
#plot average tfr of each perturbation condition for each channel
if erps == 'frn':
    baseline_t = [-1.5, -1.2]
elif erps == 'lrp':
    baseline_t = [-1.3, -1.0]
perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

if roi == 'all':
    channels = ['Fpz', 'AFz', 
                'F3', 'F1', 'Fz', 'F2', 'F4',
                'FC1', 'FCz', 'FC2', 
                'C5', 'C3', 'C1', 'Cz', 'C2', 'C4', 'C6', 
                'CP5', 'CP3', 'CP1', 'CPz', 'CP2', 'CP4', 'CP6',
                'P5', 'P3', 'P1', 'Pz', 'P2', 'P4', 'P6',
                'Oz', 'O1', 'O2']
elif roi == 'frontal':
    channels = ['Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz']
elif roi == 'central':
    channels = ['Cz', 'C3', 'C4']
elif roi == 'parietal':
    channels = ['Pz', 'P3', 'P4']
elif roi == 'visual':
    channels = ['Oz', 'O1', 'O2']

colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        data[0].plot_topo(picks = channels, baseline=(baseline_t[0], baseline_t[1]), mode="logratio", tmin=-0.25, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.2, vmax = 0.2)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels[c], baseline=(baseline_t[0], baseline_t[1]), mode="logratio", combine='mean', tmin=-1.5, tmax=0, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.2, vmax = 0.2)
        
#     save_tfr_channel_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power", channel = channels[c])

In [7]:
#plot average tfr for each of perturbation conditions/ trial types
if erps == 'frn':
    baseline_t = [-1.5, -1.2]
elif erps == 'lrp':
    baseline_t = [-1.3, -1.0]
perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

if roi == 'all':
    channels = ['Fpz', 'AFz', #frontal (cognitive)
                'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
                'Cz', 'C3', 'C4', #motor execution
                'Pz', 'P3', 'P4', #parietal areas
                'Oz', 'O1', 'O2'] #visual areas
elif roi == 'frontal':
    channels = ['Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz']
elif roi == 'central':
    channels = ['Cz', 'C3', 'C4']
elif roi == 'parietal':
    channels = ['Pz', 'P3', 'P4']
elif roi == 'visual':
    channels = ['Oz', 'O1', 'O2']
elif roi == 'medfro':
    channels = ['F1', 'Fz', 'F2',
                'FC1', 'FCz', 'FC2',
                'C1', 'Cz', 'C2']
elif roi == 'latcen':
    channels = ['C5', 'C3',
                'CP5', 'CP3', 'CP1',
                'P5', 'P3', 'P1']

colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, baseline=(baseline_t[0], baseline_t[1]), mode="logratio", combine='mean', tmin=-0.25, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.2, vmax = 0.2)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, baseline=(baseline_t[0], baseline_t[1]), mode="logratio", combine='mean', tmin=-1.5, tmax=0, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.2, vmax = 0.2)
        
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power", roi = roi)

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\early_late_aligned_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\early_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\late_rot_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\early_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\late_mir_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\early_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\late_rdm_power-tfr.h5 ...
Applying baseline correction (mode: logratio)


In [ ]:
# # subtract aligned from each perturbation condition (e.g. early_rot minus aligned)
# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# if erps == 'frn':
#     baseline_t = [-1.5, -1.2]
# elif erps == 'lrp':
#     baseline_t = [-1.3, -1.0]

# early_rot = []
# late_rot = []
# early_mir = []
# late_mir = []
# early_rdm = []
# late_rdm = []
    
# for pp in participants:
#     for pcond in range(0, len(perturb_conds)):
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = False
#         )
        
#         if pcond == 0:
#             aln_power = power.average()
#             aln_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 1:
#             early_rot_power = power.average()
#             early_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 2:
#             late_rot_power = power.average()
#             late_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 3:
#             early_mir_power = power.average()
#             early_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 4:
#             late_mir_power = power.average()
#             late_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 5:
#             early_rdm_power = power.average()
#             early_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 6:
#             late_rdm_power = power.average()
#             late_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
            
#     early_rot_contrast = early_rot_power - aln_power
#     late_rot_contrast = late_rot_power - aln_power
#     early_mir_contrast = early_mir_power - aln_power
#     late_mir_contrast = late_mir_power - aln_power
#     early_rdm_contrast = early_rdm_power - aln_power
#     late_rdm_contrast = late_rdm_power - aln_power
    
#     early_rot.append(early_rot_contrast)
#     late_rot.append(late_rot_contrast)
#     early_mir.append(early_mir_contrast)
#     late_mir.append(late_mir_contrast)
#     early_rdm.append(early_rdm_contrast)
#     late_rdm.append(late_rdm_contrast)
    

# #grand average of tfr contrasts across participants    
# alldat = mne.grand_average(early_rot, interpolate_bads=True, drop_bads=True)
# #save tfr object
# output_tfr_data(root_dir = root, data = alldat, task = 'early_rot_contrast', output = "power")
 
# alldat = mne.grand_average(late_rot, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'late_rot_contrast', output = "power")

# alldat = mne.grand_average(early_mir, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'early_mir_contrast', output = "power")

# alldat = mne.grand_average(late_mir, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'late_mir_contrast', output = "power")

# alldat = mne.grand_average(early_rdm, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'early_rdm_contrast', output = "power")

# alldat = mne.grand_average(late_rdm, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'late_rdm_contrast', output = "power")



In [8]:
#plot ALIGNED SUBTRACTED average tfr  contrasts for each of perturbation conditions/ trial types
# no baselining needed, since this was already done before subtraction
perturb_conds = ['early_rot_contrast', 'late_rot_contrast', 'early_mir_contrast', 'late_mir_contrast', 'early_rdm_contrast', 'late_rdm_contrast']

if roi == 'all':
    channels = ['Fpz', 'AFz', #frontal (cognitive)
                'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
                'Cz', 'C3', 'C4', #motor execution
                'Pz', 'P3', 'P4', #parietal areas
                'Oz', 'O1', 'O2'] #visual areas
elif roi == 'frontal':
    channels = ['Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz']
elif roi == 'central':
    channels = ['Cz', 'C3', 'C4']
elif roi == 'parietal':
    channels = ['Pz', 'P3', 'P4']
elif roi == 'visual':
    channels = ['Oz', 'O1', 'O2']
elif roi == 'medfro':
    channels = ['F1', 'Fz', 'F2',
                'FC1', 'FCz', 'FC2',
                'C1', 'Cz', 'C2']
elif roi == 'latcen':
    channels = ['C5', 'C3',
                'CP5', 'CP3', 'CP1',
                'P5', 'P3', 'P1']
    
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-0.25, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.1, vmax = 0.1)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=0, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.1, vmax = 0.1)
        
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power", roi = roi)

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\early_rot_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\late_rot_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\early_mir_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\late_mir_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\early_rdm_contrast_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\late_rdm_contrast_power-tfr.h5 ...
No baseline correction applied


In [ ]:
# # subtract EARLY from LATE for each perturbation type (e.g. late_rot minus early_rot)
# # subtract baseline aligned for each participant, then subtract early from late
# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# if erps == 'frn':
#     baseline_t = [-1.5, -1.2]
# elif erps == 'lrp':
#     baseline_t = [-1.3, -1.0]

# rot = []
# mir = []
# rdm = []

    
# for pp in participants:
#     for pcond in range(0, len(perturb_conds)):
#         data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#         freqs, n_cycles, sfreq = set_tfr_params()
        
#         power = mne.time_frequency.tfr_morlet(
#             data,
#             freqs=freqs,
#             n_cycles=n_cycles,
#             use_fft=False,
#             decim=1,
#             n_jobs=None,
#             return_itc = False,
#             average = False
#         )
        
#         if pcond == 0:
#             aln_power = power.average()
#             aln_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 1:
#             early_rot_power = power.average()
#             early_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 2:
#             late_rot_power = power.average()
#             late_rot_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 3:
#             early_mir_power = power.average()
#             early_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 4:
#             late_mir_power = power.average()
#             late_mir_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 5:
#             early_rdm_power = power.average()
#             early_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
#         elif pcond == 6:
#             late_rdm_power = power.average()
#             late_rdm_power.apply_baseline(mode='logratio', baseline=(baseline_t[0], baseline_t[1]))
            
#     early_rot_contrast = early_rot_power - aln_power
#     late_rot_contrast = late_rot_power - aln_power
#     early_mir_contrast = early_mir_power - aln_power
#     late_mir_contrast = late_mir_power - aln_power
#     early_rdm_contrast = early_rdm_power - aln_power
#     late_rdm_contrast = late_rdm_power - aln_power
    
#     rot_contrast = late_rot_contrast - early_rot_contrast
#     mir_contrast = late_mir_contrast - early_mir_contrast
#     rdm_contrast = late_rdm_contrast - early_rdm_contrast
#     #print(late_rot_contrast, early_rot_contrast, rot_contrast)
#     rot.append(rot_contrast)
#     mir.append(mir_contrast)
#     rdm.append(rdm_contrast)
    

# #grand average of tfr contrasts across participants    
# alldat = mne.grand_average(rot, interpolate_bads=True, drop_bads=True)
# #save tfr object
# output_tfr_data(root_dir = root, data = alldat, task = 'EL_rot', output = "power")
 
# alldat = mne.grand_average(mir, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'EL_mir', output = "power")

# alldat = mne.grand_average(rdm, interpolate_bads=True, drop_bads=True)
# output_tfr_data(root_dir = root, data = alldat, task = 'EL_rdm', output = "power")



In [9]:
#plot EARLY and LATE SUBTRACTED average tfr  contrasts for each of perturbation conditions/ trial types
# no baselining needed, since this was already done before subtraction
perturb_conds = ['EL_rot', 'EL_mir', 'EL_rdm']

if roi == 'all':
    channels = ['Fpz', 'AFz', #frontal (cognitive)
                'Fz', 'F3', 'F4', 'FCz', #medial frontal (motor preparatory)
                'Cz', 'C3', 'C4', #motor execution
                'Pz', 'P3', 'P4', #parietal areas
                'Oz', 'O1', 'O2'] #visual areas
elif roi == 'frontal':
    channels = ['Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz']
elif roi == 'central':
    channels = ['Cz', 'C3', 'C4']
elif roi == 'parietal':
    channels = ['Pz', 'P3', 'P4']
elif roi == 'visual':
    channels = ['Oz', 'O1', 'O2']
elif roi == 'medfro':
    channels = ['F1', 'Fz', 'F2',
                'FC1', 'FCz', 'FC2',
                'C1', 'Cz', 'C2']
elif roi == 'latcen':
    channels = ['C5', 'C3',
                'CP5', 'CP3', 'CP1',
                'P5', 'P3', 'P1']
    
colormap = 'coolwarm'
for pcond in range(0, len(perturb_conds)):
    data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
    if erps == 'frn':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-0.25, tmax=1.5, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.08, vmax = 0.08)
    elif erps == 'lrp':
        fig = data[0].plot(picks = channels, combine='mean', tmin=-1.5, tmax=0, title="%s" % (perturb_conds[pcond]), cmap = colormap, vmin = -0.08, vmax = 0.08)
    
    save_tfr_plot(root_dir=root, fig=fig[0], task = perturb_conds[pcond], output = "power", roi = roi)

Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\EL_rot_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\EL_mir_power-tfr.h5 ...
No baseline correction applied
Reading F:/Documents/Science/MirRevAdaptEEG\data/eeg/tfr_ana/frn\EL_rdm_power-tfr.h5 ...
No baseline correction applied


In [ ]:
# convert EARLY and LATE SUBTRACTED TFR to CSV
# perturb_conds = ['EL_rot', 'EL_mir', 'EL_rdm']

# for pcond in range(0, len(perturb_conds)):
#     data = load_tfr_data(root_dir=root, task = perturb_conds[pcond], output = "power")
#     df = data[0].to_data_frame()
#     df = df[['time', 'freq', 'Fpz', 'AFz', 'Fz', 'F3', 'F4', 'FCz', 'Cz', 'C3', 'C4', 'Pz', 'P3', 'P4', 'Oz', 'O1', 'O2']]
#     df_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', '%s_power_%s.csv' % (perturb_conds[pcond], erps))
#     df.to_csv(df_filename)
    

In [ ]:
# perturb_conds = ['early_late_aligned', 'early_rot', 'late_rot', 'early_mir', 'late_mir', 'early_rdm', 'late_rdm']

# for pcond in range(0, len(perturb_conds)):
#     pp_power = []
#     pp=0
#     data, ppdir = load_tfr_epochs(pp_num = pp, root_dir = root, erp_path = erps, task = perturb_conds[pcond])
#     data.compute_psd(fmin=2.0, fmax=100).plot(average=True, picks="data", exclude="bads")
# #     data.compute_psd().plot_topomap(ch_type="eeg", normalize=False, contours=0)